In [ ]:
!pip install selenium

In [93]:
import re
from pathlib import Path  

In [70]:
group_IDs=list()

with open('/Users/levtsipes/Desktop/Work/DeMAS/Bully/fb .txt','r') as file:    
    for line in file:
        #print(line)
        #print(re.findall(r"https\:\/\/www.facebook.com\/([^;]*)\n",line))
        group_IDs+=re.findall(r"https\:\/\/www.facebook.com\/([^;]*)\n",line)
group_IDs=group_IDs[:10]

In [96]:
#Final Project
#Introduction to Programming with Python and R
#Author: Budong Dong

#Objective:
#This script is used to scrap the users' ID number and their URL in Facebook groups.
#Since Facebook no longer provides API for private groups, this script uses the Selenium package to simulate web browsing activities using Edge browser.

#In order to accomplish this, necessary libraries are downloaded and imported into the current Python session.
import os
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

#Later we will write the crawled data into a csv file, so library `pandas` is downloaded and imported.
import pandas as pd
#We will also add the date/time into the filename of the written file, so `time` is downloaded and imported.
import time

#First, we need to start an Microsoft Edge webdriver.
edge_options = webdriver.EdgeOptions()
edge_options.add_argument('--disable-notifications')

#The webdriver is downloaded to local and loaded.
driver = webdriver.Edge(r'/Users/levtsipes/Downloads/edgedriver_mac64/msedgedriver',
                        options=edge_options)  # USER INPUT - file location
#Getting the website that we want to visit - here we use Facebook
driver.get('https://www.facebook.com/')

#Locating the input areas for login information, email and password
email = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name = 'email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name = 'pass']")))

#Clearing any existing content in the email and password area
email.clear()
password.clear()

#Using `send_keys` method to enter your Facebook login email and password into the areas above
email.send_keys("r.roku@yandex.ru")  # USER INPUT
password.send_keys("QuintoDeiStampi__2019")  # USER INPUT

#Login by submitting the email and password and pressing the `submit` button
login = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type = 'submit']"))).click()

#Maximizing the browser window to crawl as many lines of data as possible
driver.maximize_window()# maximize window

#Setting the number of scrolls.
n_scrolls = 15  # Each scroll has approximately 10 new users. Use (group size/10 + safety margin). Don't try this on a huge group, FB could time out

#Specifying the Name and group ID that you want to crawl data from.
#Here we are interested in a group of depression patients.

#group_names = ['Российский Союз ветеранов', 'Ветераны Афганистана ОКСВА','ВЕТЕРАНЫ АФГАНА','ТА Типичная Анорексичка']

for i in range(len(group_names)):
    try:
        #group_name = group_names[i]  # insert group name here
        group_ID = group_IDs[i]  # insert group id here

        #Getting to the specified facebook group. The URL is concatenated by coercing 'https://www.facebook.com/groups/ and the user-input group ID
        driver.get('https://www.facebook.com/' + group_ID)  # USER INPUT # Insert group number


        #Simulating the mouse scrolls to get load as much data as possible. There is a 5-second break between each scroll to avoid FB's robot detection.
        for i in range(0, n_scrolls):
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(5)

        #Extracting the URL and ID of all post senders
        #First, creat an empty list to collect users info who make posts into the interested group
        users_post = list()
        #Using the `find_elements_by_tag_name` method to find all html lines with tag `div`
        div_class_post = driver.find_elements_by_tag_name('div')
        #Looking through each div tag for our interested information
        for div in div_class_post:
            #By inspecting the HTML, the post maker information is stored under div class == "q676j6op qypqp5cg"
            if div.get_attribute('class') == "q676j6op qypqp5cg":
                #Extracting all anchor tags under this div class
                anchor_post =div.find_elements_by_tag_name('a')
                #Looking through each anchor tag for the user URL (in attribute `href`) and username (in attribute `aria-label`)
                for a in anchor_post:
                    #Extracting `a href` and a area-label`
                    a_href=a.get_attribute('href')
                    a_aria_label = a.get_attribute('aria-label')
                    #Filtering only the relevant URLs which has a fixed format of 'https://www.facebook.com/groups/' + group_ID + '/user'.
                    if a_href is not None: #and a_aria_label is not None and str(a_href).startswith('https://www.facebook.com/groups/' + group_ID + '/user'):
                        #Creating a dictionary of username-url pairs.
                        url_post = a_href
                        username_post = a_aria_label
                        user_post = [username_post, url_post]
                        #Appending the username-url pairs to the users_post list
                        users_post.append(user_post)

        #Writing the list containing username-url pairs to a csv file and indicating the content, group, written time in the file name.
        
          
        x = r'/Users/levtsipes/Desktop/Work/DeMAS/Bully/nf/POST MAKERS_result_' + re.findall(r'([^/]+)',group_ID)[0] + "_" + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + '.csv'
        filepath = Path(x)  
        filepath.parent.mkdir(parents=True, exist_ok=True)        
        df = pd.DataFrame(users_post, columns=["username","url"])
        df.to_csv(filepath)



        #Extract the URL and ID of all comment makers
        #First, creat an empty list to collect users info who make comments into the interested group
        comment_makers = list()
        #Using the `find_elements_by_tag_name` method to find all html lines with tag `div`
        div_class_comment = driver.find_elements_by_tag_name('div')
        #Looking through each div tag for our interested information
        for div in div_class_comment:
            #By inspecting the HTML, the comment information is stored under div class == "tw6a2znq sj5x9vvc d1544ag0 cxgpxx05"
            if div.get_attribute('class') == "tw6a2znq sj5x9vvc d1544ag0 cxgpxx05":
                #Extracting all span tags under this div class
                span_class_comment = div.find_elements_by_tag_name('span')
                #Looking through each span tag
                for span in span_class_comment:
                    # By inspecting the HTML, the comment maker information is stored under span class == "nc684nl6".
                    if span.get_attribute('class') == "nc684nl6":
                        #Extracting all anchor tags under this span class
                        anchor_comment = span.find_elements_by_tag_name('a')
                        #Looking through all anchor tags for url (in a href)
                        for a in anchor_comment:
                            #Extracting all urls under this tag
                            a_href = a.get_attribute('href')
                            #Filtering only the relevant urls with fixed format 'https://www.facebook.com/groups/' + group_ID + '/user'
                            if a_href is not None:
                                #Creating a dictionary of userID-url pairs
                                url_comment = a_href
                                user_ID=re.findall(r"https\:\/\/www.facebook.com\/([^;]*)\?",url_comment) #UserID is extracted from URL using regular expression
                                comment_maker = [user_ID, url_comment]
                                #Sppending the userID-URL pairs to the comment_makers list
                                comment_makers.append(comment_maker)

        #Writing the list containing userID-URL pairs to a csv file and indicating the content, group, written time in the file name.
        y = r'/Users/levtsipes/Desktop/Work/DeMAS/Bully/nf/COMMENT_MAKERS_result_' + re.findall(r'([^/]+)',group_ID)[0] + "_" + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + '.csv'
        filepath = Path(y)  
        filepath.parent.mkdir(parents=True, exist_ok=True)        
        df = pd.DataFrame(comment_makers, columns=["user_ID","url"])
        df.to_csv(filepath)

        try:
            driver.get('https://www.facebook.com/' + group_ID + 'members')  # USER INPUT # Insert group number
            print('yes!')
            members = list()


            #Simulating the mouse scrolls to get load as much data as possible. There is a 5-second break between each scroll to avoid FB's robot detection.
            for j in range(0, n_scrolls):
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
                time.sleep(5)



            #Using the `find_elements_by_tag_name` method to find all html lines with tag `div`
            div_class_member = driver.find_elements_by_tag_name('div')
            #Looking through each div tag for our interested information
            for div in div_class_member:
                #By inspecting the HTML, the comment information is stored under div class == "tw6a2znq sj5x9vvc d1544ag0 cxgpxx05"
                if div.get_attribute('class') == "qzhwtbm6 knvmm38d":
                    #Extracting all span tags under this div class
                    span_class_member = div.find_elements_by_tag_name('span')
                    #print(span_class_member)
                    #Looking through each span tag
                    for span in span_class_member:
                        # By inspecting the HTML, the comment maker information is stored under span class == "nc684nl6".
                        if span.get_attribute('class') == "nc684nl6":
                            #Extracting all anchor tags under this span class
                            anchor_member = span.find_elements_by_tag_name('a')
                            #print(anchor_member)
                            #Looking through all anchor tags for url (in a href)
                            for a in anchor_member:
                                #Extracting all urls under this tag
                                a_href = a.get_attribute('href')
                                #print(a_href)
                                #Filtering only the relevant urls with fixed format 'https://www.facebook.com/groups/' + group_ID + '/user'
                                if a_href is not None:
                                    #Creating a dictionary of userID-url pairs
                                    url_member = a_href
                                    user_ID=re.findall(r'https\:\/\/www.facebook.com\/'+group_ID+r"user\/(.*)\/",url_member) #UserID is extracted from URL using regular expression
                                    actual_member = [user_ID, url_member]
                                    #Sppending the userID-URL pairs to the comment_makers list
                                    members.append(actual_member)
                                    
            if len(members)!=0:

                z = r'/Users/levtsipes/Desktop/Work/DeMAS/Bully/nf/MEMBERS_result_' + re.findall(r'([^/]+)',group_ID)[0] + "_" + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + '.csv'
                filepath = Path(z)  
                filepath.parent.mkdir(parents=True, exist_ok=True)        
                df = pd.DataFrame(comment_makers, columns=["user_ID","url"])
                df.to_csv(filepath)

        except:
            pass
    except:
        print('exception:', group_ID)
    
    
    os.system('say "group is finished"')

os.system('say "Code is finished"')



<ipython-input-96-79ca214c6e3a>:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(r'/Users/levtsipes/Downloads/edgedriver_mac64/msedgedriver',
<ipython-input-96-79ca214c6e3a>:76: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  div_class_post = driver.find_elements_by_tag_name('div')
<ipython-input-96-79ca214c6e3a>:112: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  div_class_comment = driver.find_elements_by_tag_name('div')


yes!


<ipython-input-96-79ca214c6e3a>:159: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  div_class_member = driver.find_elements_by_tag_name('div')


0